In [1]:
import numpy as np
import pandas as pd

In [2]:
name_dicts = []

def build_dict():
    t1_names = ['周', '魯', '齊', '衛', '宋', '鄭', '陳', '蔡', '楚', '吳', '晉', '越']
    t2_names = ['周', '魯', '齊', '宋', '晉', '魏', '趙', '韓', '鄭', '楚', '越', '秦']
    t3_names = ['周', '秦', '魏', '韓', '趙', '楚', '越', '齊', '燕', '宋', '魯', '滕']
    t4_names = ['周', '秦', '魏', '韓', '趙', '楚', '齊', '宋', '燕']
    t5_names = ['周', '秦', '魏', '韓', '趙', '楚', '齊', '宋', '燕']

    names = [t1_names, t2_names, t3_names, t4_names, t5_names]

    for n in names:
        # state number starts from 1, not 0
        name_dicts.append(dict(zip(range(1,len(n)+1), n)))

build_dict()

for d in name_dicts:
    # d.values() returns an iterator, which will cause 'RuntimeError: dictionary changed size during iteration'
    # error when setting default values in the next step. Change it into a static list to avoid the problem.
    values = list(d.values())
    
    # Set up default values for non-existing keys.
    # Here we just use the key it self as output if the key is in the tx_names list.
    for v in values:
        d.setdefault(v, v)

In [3]:
# File names for raw and processed data
csv_fnames = ['先秦諸子繋年-通表第一.csv',
              '先秦諸子繋年-通表第二.csv',
              '先秦諸子繋年-通表第三.csv',
              '先秦諸子繋年-通表第四-上.csv',
              '先秦諸子繋年-通表第四-下.csv']

In [4]:
src_data_path = '../../db/csv_raw/'

def to_int(value):
    if type(value)==str:
        if value in [str(x) for x in list(range(20))]:
            return int(value, 10)
        elif value=='':
            return np.NaN
        else:
            return value
    else:
        return value
    

# Convert dictionary for converting digit strings to integers.
convert = dict(zip(['year', 'state', 'name', 'event', 'notes'], [to_int, None, None, None, None]))

dfs = []

for fname in csv_fnames:
    dfs.append(pd.read_csv(src_data_path + fname, header=0, converters=convert))

In [ ]:
for i, df in enumerate(dfs): 
    df.loc[:, 'year'].fillna(method='ffill', inplace=True)
    df.loc[:, 'state'].fillna(method='ffill', inplace=True)
    df.loc[:, 'name'].fillna(method='ffill', inplace=True)     
    
    df['state'] = df.loc[:, 'state'].map(name_dicts[i])

In [ ]:
# Check if there's any unresolved 'year', 'state', or 'name'
nans = []
for df in dfs:
    nans.append(df.iloc[:, 0:2].isnull().sum().sum())

# nans should be 0 for all 5 tables.
nans

In [ ]:
# Example for selecting all cells that contains '生' in dfs[0].event column.

# In pandas, selecting all cells that contains '生' from a column will produce a Boolean column except that
# all the NaN cell originally in the column will be NaN in the result. I.e., if a cell was NaN, although it's
# not '生', the test for '生' will not produce False, but NaN.

# Such column, which contains Boolean and NaN values, cannot be used to select cells.

# To fix this, use .notnull() to select the original column which turns NaN cells into Flase. Selecting
# cells that contains '生' will produce False for the original NaN cells.
dfs0event = dfs[0].event[dfs[0].event.notnull()]
dfs0event
dfs0event[dfs0event.str.contains('生')]

In [7]:
# Destination folder for processed data
dest_data_path = '../../db/csv/'

In [8]:
# Save the processed 通表第一, 二, 三
for i, df in enumerate(dfs):
    if i<=2:
        df.to_csv(dest_data_path + csv_fnames[i], index=False)

In [9]:
# Combine processed 通表第四-上, 下, and save to 通表第四
df4 = pd.concat([dfs[3], dfs[4]])
df4.to_csv(dest_data_path + '先秦諸子繋年-通表第四.csv', index=False)

In [10]:
# Combine all 通表 and save to tongbiao.csv (通表)
df_combined = pd.concat(dfs)
df_combined.to_csv(dest_data_path + 'tongbiao.csv', index=False)